In [142]:
#for aoi in list_aois[:1]:
    #shape = shapely.to_geojson(tiles.loc[aoi,"geometry"])
    #f = write_json(shape)
#    tile = tiles[tiles['random']==aoi]
    #print(type(tile))
#    tile.to_file('temp.geojson')
    

<class 'geopandas.geodataframe.GeoDataFrame'>


This code works through the GEDI subsetter version 0.6.0. This converts a geojson of tiles over a study area to temp geojson files to feed into the subsetter

In [1]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')


In [4]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import shapely

In [5]:
#Load in Tiled AOI
tiles = gpd.read_file("https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/WestAfricaGrid.geojson")

In [12]:
import os

result_folder = "../aoi/WestAfricaGrid"

In [4]:


if not os.path.exists(result_folder):
    os.makedirs(result_folder)

for rownum, subframe in tiles.groupby("random"):
    filename = os.path.join(result_folder, f"{rownum}.geojson")
    subframe.to_file(filename, driver='GeoJSON')

len(os.listdir(result_folder))

151

In [156]:
#os.listdir(result_folder)

In [61]:
#Set index to the correct column name, in this case "random"
#tiles.set_index('random',inplace=True)
#print(tiles.head())

In [62]:
#for col in tiles.columns:
#    print(col)

In [63]:
#tiles['random_value'] = tiles.index

In [64]:
#Test a single tile
#tiles.loc[0.6855097362903453]

Use "random" column for ID to generate features to tile over

In [65]:
#List out the individual aois
#list_aois = tiles.index.unique().tolist() #tiles[tiles.columns[1]].values.tolist()
#print(list_aois)

In [66]:
#aoiTest=shapely.to_geojson(tiles.loc[0.6855097362903453,"geometry"])
#print(aoiTest)

In [6]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [10]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [41]:
#Import geojson and tempfile for creating temp files for the subsetter

#from geojson import Point, Feature, FeatureCollection, dump

#import tempfile

In [23]:
#Set up input dictionary and import backoff to display run results
#First part of the for loop uses the "random" column to search through the tiles geojson to 
#create temp files one at a time to set the AOI
#################################

for aoi in os.listdir(result_folder):
    if aoi.endswith("geojson"):
        inputs = dict(
           aoi=f"https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/aoi/WestAfricaGrid/{aoi}",
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 0,
           temporal="-",
           output=f"{aoi}"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.6.0",
            queue="maap-dps-worker-32gb",
            username="abarenblitt",
            **inputs
        )

        job_id = result["job_id"]
        print(job_id)
        
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

4933834f-004f-4a79-ab5f-fbae86e57e51
2b7a982e-ee5d-4986-a443-8e004e1eaffd
42a91f41-79ae-431b-9f60-ea2ed4c82eff
f65af8fe-78de-4f89-94fa-f1136a69a19f
25bd7ce9-558d-44df-b4a8-cabebb9dd55c
488747dd-b724-4440-8558-e24a098919b8
ed4fdcad-d917-486d-8f94-0c6c6025d57c
96b124fa-633b-4bd0-979c-d43e634101b0
d3047dcf-27ef-4e5d-8ec0-6998ef42d233
6f1e95f1-6e83-461a-bb5a-af2dc7cf6926
a3a905fe-08b0-4252-835c-a24b482a5bc3
06f7793e-e1f0-4d29-b399-b47fa844e7a3
c61ca323-6a53-42bf-904b-5794d6f39f99
e18f47ea-ac78-410b-aeb8-9a6c7de7817e
96fa653f-c865-4373-85f7-8a14f0eb5bef
a855fdee-cbe1-4a57-9192-3aea4caf25a5
37093f47-bbdb-48ba-ab78-9eede4c7a47a
29051a42-6a7e-4546-a7fb-34005aa6e59c
c5c65f7c-5541-459b-a26d-f6315a7c1a2f
880c1ebc-abcc-4bdf-b6f3-64ad2f01a4f1
eb5031c5-4a99-4094-a1ae-980d3dde19da
f5231bc0-ad7a-4f26-8223-4789b88ed0e0
49516920-6e4a-434d-b489-44165bd1a449
9d48486b-099e-4d0f-a67b-4a07702f8aba
b2c9b438-859a-49fe-b62e-876abe52ea1a
b4c405f5-8daf-4786-89c1-f73d00517564
8c1ffe8d-ccfe-43d1-b796-320126260580
e

In [19]:
job_id = "11414950-c5c9-4b99-9c1c-c9df02ba1923"

job_status

'Accepted'

In [20]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

In [25]:
import os
import subprocess
root_dir = '/projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30'
out_dir = '/projects/my-private-bucket/dps_output/gedi-subset/0.6.0/WestAfricaGrid'
for dirName, subdirList, fileList in os.walk(root_dir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        if fname.endswith('.gpkg'):
            print(fname)
            subprocess.call('cp ' + os.path.join(dirName, fname) + ' ' + os.path.join(out_dir, fname),shell=True)

Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/40
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/40/31
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/40/31/344731
0.0008195283710246137.gpkg
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/41
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/41/09
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/41/09/208212
0.07476782962538708.gpkg
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/44
Found directory: /projects/my-private-bucket/dps_output/gedi-subset/0.6.0/2023/06/30/14/44/52
Found directory: /